导入模块

In [30]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [31]:
tf.__version__

'1.2.1'

加载数据集

In [32]:
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


设置超参数

In [33]:
batch_size=100
n_batch=mnist.train.num_examples//batch_size

初始化权重 卷积层W.shape = [filter_height,filter_width,in_channels,out_channels]

        全连接层[,]

In [34]:
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

初始化偏置b = [卷积核的个数]

In [35]:
def bias_variables(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))

卷积层

In [36]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

池化层

In [37]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

定义占位符

In [38]:
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

改变图片的shape匹配模型的输入[batch,in_height,in_width,in_channels]

In [39]:
x_image = tf.reshape(x,[-1,28,28,1])

定义第一层卷积层的权重

In [40]:
W_conv1 = weight_variable([5,5,1,32])

定义第一层卷积层的偏置

In [41]:
b_conv1 = bias_variables([32])

图片数据与权值向量进行卷积，再加上偏置值，然后使用relu激活函数

In [45]:
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)

池化

In [46]:
h_pool1 = max_pool_2x2(h_conv1)

定义第二层卷积层的权重

In [42]:
W_conv2 = weight_variable([5,5,32,64])

定义第二层的偏置

In [44]:
b_conv2 = bias_variables([64])

tensor流入第二层卷积层

In [47]:
h_conv2 = conv2d(h_pool1,W_conv2)

池化

In [48]:
h_pool2 = max_pool_2x2(h_conv2)

定义全连接神经网络的第一层网络节点

In [49]:
W_fc1 = weight_variable([7*7*64,1024])

In [50]:
b_fc1 = bias_variables([1024])

把池化层的输出扁平化为一维以匹配全连接神经网络的输入

In [51]:
h_pool_2_flat = tf.reshape(h_pool2,[-1,7*7*64])

tensor flow到该结点

In [52]:
h_fc1 = tf.nn.relu(tf.matmul(h_pool_2_flat,W_fc1)+b_fc1)

选择性激活部分神经元

In [53]:
keep_prob = tf.placeholder(tf.float32)

In [57]:
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

定义全连接神经网络第二层网络节点

In [55]:
W_fc2 = weight_variable([1024,10])

In [56]:
b_fc2 = bias_variables([10])

tensor flow到该结点

In [58]:
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

反向传播

交叉熵--损失函数

In [60]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

使用AdamOptimizer进行优化

In [61]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

计算准确率(不熟悉)

In [62]:
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))

In [63]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

进行训练

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter"+str(epoch)+",Testing Accuracy = "+str(acc))